<img src="uva_seal.png">  

## Spark SQL and DataFrames

### University of Virginia
### DS 5110: Big Data Systems
### Last Updated: January 17, 2026

---  

### Sources 

Learning Spark, Chapter 9: Spark SQL

https://spark.apache.org/docs/latest/sql-programming-guide.html

https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning

Demonstration of several useful DataFrame operations:  
https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html

### OBJECTIVES
- Introduction to Spark SQL, the interface for working with structured and semistructured data
- Introduce DataFrames and show basic functionality
- Implement queries using Spark SQL
- Execute tasks including filtering and aggregations using Spark DataFrames

### CONCEPTS AND FUNCTIONS
- Spark SQL
- Temp table
- Dataset and DataFrame
- Aggregations with groupBy()

---  

### 1. Overview

Two important ways of working with big data in Spark: 

- Through Spark SQL
- Using DataFrames

They also interoperate

### 2. SQL in Ten Seconds (tongue in cheek)


SQL is a structured query language used to communicate with relational databases.  
Commands include CREATE, SELECT, UPDATE, ALTER, INSERT INTO, DROP, DELETE.  
This course will use SELECT.

### 3. Spark SQL Capabilities:

- Load data from various structured formats including JSON, Hive, Parquet  
- Query data using SQL inside Spark or from external tools that connect to Spark (e.g., `Tableau`) 
- Spark SQL integrates between SQL and Python/Java/Scala/R code. Can do things like join RDDs and SQL tables.

### 4. Note on Spark SQL Development

- Spark SQL has been heavy development area in new releases 
- As the module involves massive amounts of data, optimizing operations is valuable
 
---

### 5. Dataset and DataFrame

- A Dataset is a distributed collection of data   
- A Dataset can be constructed from JVM objects
- It can be manipulated using functional transformations (`map()`, `flatMap()`, `filter()`, etc.)  
- A DataFrame is a Dataset organized into named columns   

In practice, you will be thinking in terms of `DataFrames`, and not `Datasets`.  

For users familiar with dataframes from R and Python, they are similar, yet with operations distinct to Spark.  

As an example, adding a new column to a DataFrame is executed using `withColumn()`.  

This may feel more formal compared to R and Python.  

Additionally - when compared to R and Python: 

**The Spark DataFrame is built up from RDDs.    
It inherits the properties of distributed data, lazy execution, DAGs.**  

DataFrames can be constructed from a wide array of sources such as:  
structured data files, tables in Hive, external databases, or existing RDDs.  

The DataFrame API is available in Scala, Java, Python, and R. 

---

### 6. DataFrames vs RDDs  

When is it better to use DataFrames, and when is it better to use RDDs?  

Here are some recommendations:   

- In general, most work can be done with DataFrames  

- Use DataFrames to use high-level expressions, to perform SQL queries to explore the data, and to gain columnar access.

**When to use DataFrames**

- If you are thinking about the data by field names, you probably want the data in a DataFrame

- For machine learning and building predictive models, DataFrames are recommended.  
  You will be exploring the data by column, and building features from the columns of data.

**When to use RDDs**
  
- RDDs can be useful to perform low-level transformations and actions on unstructured data.  
  For example, filtering strings and performing other simple transformations on text is best done with RDDs.  
  In these cases, the analyst doesn't care about field names, and there is no need to impose schema on the data.  

- Use RDDs when you want to manipulate the data with functional programming constructs rather than domain specific expressions.

---

### 7. Creating a DataFrame

There are multiple ways to do this:
- use a function such as `read.csv()` to read data from files into DataFrames (most common)
- pass data to `createDataFrame()` with schema
- conversion from RDD using `toDF()`

**Example 1: Create DataFrame from RDD using `toDF()`**

---  
```
# import modules 
from pyspark.sql import Row

# Map the RDD to a DF

df = rdd.map(lambda line: Row(longitude=line[0], 
                              latitude=line[1], 
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5], 
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()
```
---  

**Example 2: Create DataFrame by passing data and schema to `createDataFrame()`**

In [ ]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# set up the session
spark = SparkSession.builder.getOrCreate()

# create some data; list of tuples
data = [
    (0, "ChatGPT is all the rage"),
    (1, "Google released BARD to compete"),
    (2, "What does AWS think about this?")
]

# define schema; each field holds (name, data type, nullable)
# for large number of fields, best to automate schema construction
schema = StructType([StructField('id', IntegerType(), False), 
                     StructField('sentence', StringType(), False)])

# create df by passing data, schema
sentenceDataFrame = spark.createDataFrame(data, schema)

# print first few records
sentenceDataFrame.show(3, False)

# print data type
print(type(sentenceDataFrame))

**Example 3: Create a DataFrame from some JSON data with spark.read()**  
(For an example of JSON data see: http://json.org/example.html)


In [ ]:
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Read data in json format
df = spark.read.json("../data/people.json")

# Displays the content of the DataFrame to stdout
df.show()

**Creating an RDD from a DataFrame**

This is very simple: `df.rdd`

Here we convert our df containing sentences:

In [ ]:
sentence_rdd = sentenceDataFrame.rdd
print(sentence_rdd.take(2))
print(type(sentence_rdd))

---

### 8. Some Useful DataFrames Operations

Next we explore subsetting, filtering, and aggregation among others.

In [ ]:
# Read data in json format
df = spark.read.json("../data/people.json")
df.show()

### Selecting

There are three ways to select columns and we show them all. Find your favorite!

- Bracket operator
- `col()` method
- Dot operator (my favorite)

We see them below

### Filtering

Keep records where age > 21

In [ ]:
# bracket operator
df.filter(df['age'] > 21).show()

In [ ]:
# dot operator
df.filter(df.age > 21).show()

In [ ]:
# column operator
from pyspark.sql.functions import col

df.filter(col('age') > 21).show()

Keep records subject to filters on name, then sort

In [ ]:
from pyspark.sql.functions import asc

# alternatively using df.name instead of col("name")
df.filter((df.name == "Andy") | (df.name == "Michael")).sort(asc("name")).show()

Fetch records with age *null*

In [ ]:
df.filter(col("age").isNull()).show() 

Fetch records with age *not null*

In [ ]:
df.filter(col("age").isNotNull()).show() 

### where() is equivalent to filter()

In [ ]:
df.where((df.name == "Andy") | (df.name == "Michael")).sort(asc("name")).show()

### Impute missing with 0 (just for illustration; not a great idea for this data)

In [ ]:
df.fillna(0).show()

### Summarize the age field

In [ ]:
df.describe("age").show()

---

### 9. Spark SQL Queries

The interface of SQL and DataFrames.    
To write SQL queries against DataFrames, first register DF as a `SQL temp view`, and then write the query.  
A temp view does not persist; it only lasts until the session ends.


**Example of SQL Query against DataFrame**

In [ ]:
# register DataFrame as temp view with name "people"
df.createOrReplaceTempView("people")

# query the view
sqlDF = spark.sql("SELECT * FROM people where name == 'Andy'")
sqlDF.show()

---

### 10. Aggregate on columns

SQL functions can be loaded from this library: `pyspark.sql.functions`

We will load some stock data to demonstrate aggregation.

In [ ]:
# import data types and functions
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, FloatType
from pyspark.sql import functions as F

# set up schema
stock_schema = StructType([StructField('date',TimestampType(),False),
                           StructField('ticker',StringType(),False),
                           StructField('close',FloatType(),False),
                           StructField('adjusted_close',FloatType(),False),
                           StructField('volume',LongType(),False),
                         ])

In [ ]:
# Read in stock data. Source: Yahoo! finance
DATAPATH_STOCKS = '../data/amzn_msft_prices.csv'

df_stx = spark.read.csv(DATAPATH_STOCKS, header=True, schema=stock_schema)
df_stx.show()

In [ ]:
# check the schema
df_stx.printSchema()

We will calculate some statistics on each stock:
- minimum closing price
- maximum closing price
- minimum volume
- maximum volume

**IMPORTANT NOTE**  
Do NOT use loops to aggregate data. Loops are run sequentially and do not harness parallelization.  
Using the `groupBy()` method will do the job using parallelization.  
It follows the split, apply, combine method:

- *Split* by one or more grouping variables
- *Apply* a function to the data parts
- *Combine* the result from each grouping level

In [ ]:
agg_df = df_stx.groupBy("ticker").agg(F.min("close"), F.max("close"), F.min("volume"), F.max("volume"))
agg_df.show()

---

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Given the stock dataframe, use Spark SQL to select all AMZN records. 

In [ ]:
# SOLUTION

# register stock dataFrame as temp view with name "stocks"
df_stx.createOrReplaceTempView("stocks")

# query the view
sqlDF = spark.sql("SELECT * FROM stocks where ticker == 'AMZN'")
sqlDF.show()

2) Given the stock dataframe, do an aggregation to compute minimum, mean, and maximum adjusted close for each stock.

In [ ]:
df_stx.groupBy("ticker").agg(F.min("adjusted_close"), F.mean("adjusted_close"), F.max("adjusted_close")).show()

3) Select the date, ticker, and adjusted_close columns, saving this data as a parquet file.

In [ ]:
df_stx.select("date","ticker","adjusted_close").write.save("stocks.parquet", format="parquet")

4) Load the parquet file into a new dataframe and verify that things look correct.

In [ ]:
test = spark.read.load("stocks.parquet")
test.show()

---

### 11. Summary

You should now have a basic understanding of:

- Spark SQL
- DataFrames
- When to use DataFrames vs RDDs
- How to use some of the common transformations on DataFrames  

Given practice, you will gain comfort in selecting and processing data with Spark SQL and DataFrames, which is essential.

Additionally, you should have some sense of when DataFrames are preferred over RDDs, and vice versa.
